## **Extract features from image**

### **Import necessary libraries**

In [2]:
import os 
import sys

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

import warnings 
warnings.filterwarnings('ignore')

from PIL import Image 

import torch
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
from torch.autograd import Variable


In [3]:
current_cwd = os.getcwd()
lst = current_cwd.split('\\', 2)
new_cwd = '/'.join(lst[:2])
os.chdir(new_cwd)
sys.path.append(new_cwd + '/code')

In [4]:
IMAGES_PATH = 'data/flickr8k/Images'
CAPTIONS_PATH = 'data/flickr8k/captions.txt'

data = pd.read_csv(CAPTIONS_PATH)
data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .


In [5]:
images_data = data.drop_duplicates(subset=['image'])
images_data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
5,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
10,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front o...
15,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
20,1007129816_e794419615.jpg,A man in an orange hat starring at something .
...,...,...
40430,990890291_afc72be141.jpg,A man does a wheelie on his bicycle on the sid...
40435,99171998_7cc800ceef.jpg,A group is sitting around a snowy crevasse .
40440,99679241_adc853a5c0.jpg,A grey bird stands majestically on a beach whi...
40445,997338199_7343367d7f.jpg,A person stands near golden walls .


### **Load the InceptionV3**

In [6]:
inceptionv3 = models.inception_v3(pretrained=True)

inceptionv3.eval()

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:39<00:00, 2.76MB/s] 


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [8]:
layer = inceptionv3._modules.get('dropout')

In [9]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [12]:
def get_vector_inceptionv3(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 2048
    my_embedding = torch.zeros(2048)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    inceptionv3(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [13]:
inceptionv3_image = {}
cnt = 0
for img in images_data.image:
    inceptionv3_image[img] = get_vector_inceptionv3(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [14]:
inceptionv3_features = images_data

In [15]:
inceptionv3_features['image features'] = inceptionv3_features.image.map(inceptionv3_image)

In [16]:
inceptionv3_features = inceptionv3_features[['image', 'image features']].reindex()
with open('Image features/inceptionv3.csv', mode='w', encoding='utf-8') as file: 
    inceptionv3_features.to_csv(file, index=False)